# Welcome!

In this notebook...

In [1]:
%pip install opvious


[notice] A new release of pip available: 22.3.1 -> 23.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


+ Platform components

+ Install Python SDK
+ Create an optimization model
  + definitions
  + annotations
+ Show how to solve
  + using client
  + uploading formulation, then using cURL
  + long-running solves, queue
+ Local API server
  + Starting one
  + Configuring client

In [6]:
import opvious.modeling as om

class BinPacking(om.Model):
    items = om.Dimension()  # All items to bin
    weight = om.Parameter.non_negative(items)  # Weight per item
    bins = om.interval(1, om.size(items), name="B")  # Available bins
    max_weight = om.Parameter.non_negative()  # Maximum weight for each bin
    assigned = om.Variable.indicator(bins, items)  # Bin to item assignment
    used = om.fragments.ActivationVariable(assigned, projection=1)  # 1 if a bin is used

    @om.constraint
    def each_item_is_assigned_once(self):
        for i in self.items:
            yield om.total(self.assigned(b, i) for b in self.bins) == 1

    @om.constraint
    def bin_weights_are_below_max(self):
        for b in self.bins:
            bin_weight = om.total(self.weight(i) * self.assigned(b, i) for i in self.items)
            yield bin_weight <= self.max_weight()

    @om.objective
    def minimize_bins_used(self):
        return om.total(self.used(b) for b in self.bins)

In [8]:
BinPacking().specification()

<div style="margin-top: 1em; margin-bottom: 1em;">
<details open>
<summary style="cursor: pointer; text-decoration: underline; text-decoration-style: dotted;">BinPacking</summary>
<div style="margin-top: 1em;">
$$
\begin{align*}
  \S^d_\mathrm{items}&: I \\
  \S^p_\mathrm{weight}&: w \in \mathbb{R}_+^{I} \\
  \S^a&: B \doteq \{ 1 \ldots \# I \} \\
  \S^p_\mathrm{maxWeight}&: w^\mathrm{max} \in \mathbb{R}_+ \\
  \S^v_\mathrm{assigned}&: \alpha \in \{0, 1\}^{B \times I} \\
  \S^v_\mathrm{used}&: \psi \in \{0, 1\}^{B} \\
  \S^c_\mathrm{usedActivates}&: \forall b \in B, i \in I, \psi_{b} \geq \alpha_{b,i} \\
  \S^c_\mathrm{eachItemIsAssignedOnce}&: \forall i \in I, \sum_{b \in B} \alpha_{b,i} = 1 \\
  \S^c_\mathrm{binWeightsAreBelowMax}&: \forall b \in B, \sum_{i \in I} w_{i} \alpha_{b,i} \leq w^\mathrm{max} \\
  \S^o_\mathrm{minimizeBinsUsed}&: \min \sum_{b \in B} \psi_{b} \\
\end{align*}
$$
</div>
</details>
</div>

In [ ]:
import opvious.modeling as om

class BudgetAllocation(om.Model):
    projects = om.Dimension()
    budget = om.Parameter.non_negative()
    cost = om.Parameter.non_negative(projects)
    value = om.Parameter.non_negative(projects)
    selected = om.Variable.indicator(projects)

    @om.constraint
    def within_budget(self):
        yield om.total(self.selected(p) * self.cost(p) for p in self.projects) <= self.budget()

    @om.objective
    def maximize_value(self):
        return om.total(self.selected(p) * self.value(p) for p in self.projects)
    
model = BudgetAllocation()